![](./crawlerNotesFig/22.png)  
This article is an example about how to login to a website by using python module Requests.Step by step,I make a detailed description about how to analyse the response and request.Moreover,first part also describes how to guess password by using a brute force crack method.By using a bottom-up approach,the process is clear and efficient.

In [1]:
#import the required packages
import requests
import cookielib
import urllib2
from pytesseract import pytesseract
from PIL import Image
import cStringIO
from bs4 import BeautifulSoup
import json
import pandas as pd
import numpy as np
from datetime import datetime
username,password=open(r'C:\account.txt').read().split('\n') #read my username and password

First,we need to start a session by using `requests.session()`

In [2]:
session=requests.session()

Next,we need to use the first login page to get some cookies.Open the login page and open the developer tools we can see the page as follows:
![](./crawlerNotesFig/1.png)

open the first file in developer tools named 
>'login?service=http%3A%2F%2Fmy.whu.edu.cn%2F'  

the details are shown as:
![](./crawlerNotesFig/2.png)

From the figure above,we know that the request method is 'GET',so `session.get()` is used.The request headers informations are also given in this figure.

In [3]:
url1=r'http://cas.whu.edu.cn/authserver/login?service=http://my.whu.edu.cn/'
header1={
    'Host':'cas.whu.edu.cn',
    'User-Agent':'Mozilla/5.0 (Windows NT 10.0; WOW64; rv:54.0) Gecko/20100101 Firefox/54.0',
    'Referer':'http://my.whu.edu.cn/'
}

r1=session.get(url1,headers=header1)
print r1.status_code

200


Next,we begin to analyse the login process.Be sure to tick the buttion of 'Enable persistent logs' in setting of developer tool,since,usually,if log is not perserved,the developer tool we only show the files loaded after login,that is,the process of login will be passed.Now,click the login button and use developer tool to monitor this process,we can find that a few new files haved been loaded.Open the one named 
>'login?service=http%3A%2F%2Fmy.whu.edu.cn%2F'.  

![](./crawlerNotesFig/8.png)
![](./crawlerNotesFig/4.png)

From the headers we know its request method is 'POST',and there are also some infomations about the request headers.Now we check its params.The form data to post is here.It may be strange that the form data contains several other items besides username and password.However,tate it easy,these items can certainly be found in the files having been loaded.So,our next goal is to find the values of these 'strange items'.  
![](./crawlerNotesFig/5.png)

The request method is 'POST',so `session.post()` will be used.On the bottom,the form can be found.It is strange that the form data contains several other items besides username and password.However,take it easy,these items can certainly be found in the files having been loaded.So,in this situation,our goal is to find the values of 'lt','dllt','execution',and '_eventld'.  

Next,we can copy one of the values in these items,for example.Suppose we have copied the value of lt,that is *'LT-1672447-NMdS9bi0sdGIcPQbbvunQSi2nkUczq1499734795811-mrXp-cas'*,now we open the response of the files prior *'login?service=http%3A%2F%2Fmy.whu.edu.cn%2F'*one by one to find the copied text.There is a trick that we can neglect several type files,such as png,script,gif and so on.

Use CLTR+F to find the copied text in the 'page source' we have opened.We can find the copied text in the first file named*'
login?service=http%3A%2F%2Fmy.whu.edu.cn%2F'*.And the other 'strange items' are also here.
![](./crawlerNotesFig/6.png)

So,before using `session.post()` to post the form data,we need to get the values of those 'strange items' by parsing the html of the file mentioned above.

In [4]:
#test whether the values are in the html
'_eventId' in r1.content

True

In [5]:
#parse the target values
soup1=BeautifulSoup(r1.content,'lxml')
form=soup1.find_all("form",{'id':'casLoginForm'})[0]

lt=form.find('input',{'name':'lt'})['value']
dllt=form.find('input',{'name':'dllt'})['value']
execution=form.find('input',{'name':'execution'})['value']
_eventId=form.find('input',{'name':'_eventId'})['value']
rmShown=form.find('input',{'name':'rmShown'})['value']
print lt
print dllt
print execution
print _eventId
print rmShown

LT-1824183-LlVwTM1liC2KfZ3fEmlez9RMishMeU1499745204808-iHnm-cas
userNamePasswordLogin
e1s1
submit
1


There are all here.Now,let's post the data.

In [6]:
url2=r'http://cas.whu.edu.cn/authserver/login?service=http://my.whu.edu.cn/'
header2={
    'Host': 'cas.whu.edu.cn',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64; rv:54.0) Gecko/20100101 Firefox/54.0',
    'Referer':'http://cas.whu.edu.cn/authserver/login?service=http%3A%2F%2Fmy.whu.edu.cn%2F'
}

postdata={'username':username,
          'password':password,
          'lt':lt,  #these items can be get from html
          'dllt':dllt,
          'execution':execution,
          '_eventId':_eventId,
          'rmShown':rmShown
          }

r2=session.post(url2,data=postdata,headers=header2)

In [7]:
print r2.status_code

200


Year,it's right.We can now begin to crawl the data!  
Checking the reponse of the files one by one,we find the data we want are all in the file named '/' as following:
![](./crawlerNotesFig/10.png)

Get it.

In [8]:
url3='http://my.whu.edu.cn/'
header3={
    'Host':'my.whu.edu.cn',
    'User-Agent':'Mozilla/5.0 (Windows NT 10.0; WOW64; rv:54.0) Gecko/20100101 Firefox/54.0',
    'Referer':'http://cas.whu.edu.cn/authserver/login?service=http%3A%2F%2Fmy.whu.edu.cn%2F'
}

r3=session.get(url3,headers=header3)

ConnectionError: ('Connection aborted.', error(10060, ''))

Here is something wrong.Nevertheless,nothing to worry about.   
***Several aspect:***  
* **Is the request method right?***
* ***Is there something missed in request headers?***
* ***Is there typo error in url or request headers,and form data?***
* ***Are the Request cookies right?***  


Having made sure that there is no problem in the first three aspects,we begin to analyse the request cookies.

Now let's begin to analyse the cookies from the beginning.Focusing our attention on the 'Cookies' to find whether whether the 'Cookies' if different from those in the last file.If it is different,find whether there is an item named 'Set-Cookies' upward beginning from the last file.

we can find the cookie is changed in file named *'?ticket=ST-376925-aOl7OVV6iBZnl1dQXYZk1499734811083-rvbW-cas'* and we can find the cookie is from the file named *'login?service=http%3A%2F%2Fmy.whu.edu.cn%2F'*
![](./crawlerNotesFig/15.png)
![](./crawlerNotesFig/16.png)

For cookies in '/',it is the same story.We can find the new cookies are from *'/?ticket=ST-376925-aOl7OVV6iBZnl1dQXYZk1499734811083-rvbW-cas'*.
![](./crawlerNotesFig/17.png)
![](./crawlerNotesFig/18.png)

Now it is clear that we can using a bottom-up method to analyse the crawling process.  

To get the target data,we need get the html named  
>'/'  

and we need cookies to get it,so we have to get cookies from
>'/?ticket=ST-376925-aOl7OVV6iBZnl1dQXYZk1499734811083-rvbW-cas'  

then 
>'login?service=http%3A%2F%2Fmy.whu.edu.cn%2F' and 'login?service=http%3A%2F%2Fmy.whu.edu.cn%2F'.  


The codes are as follows:

In [29]:
import requests
import cookielib
import urllib2
from pytesseract import pytesseract
from PIL import Image
import cStringIO
from bs4 import BeautifulSoup
import json
import pandas as pd
import numpy as np
from datetime import datetime

username,password=open(r'C:\Python27\zht\whu\account').read().split('\n')

In [30]:
session=requests.session()

url1=r'http://cas.whu.edu.cn/authserver/login?service=http%3A%2F%2Fmy.whu.edu.cn%2F'
headers1={
    'Host':'cas.whu.edu.cn',
    'User-Agent':'Mozilla/5.0 (Windows NT 10.0; WOW64; rv:54.0) Gecko/20100101 Firefox/54.0'
    }
r1=session.get(url1,headers=headers1)

In [31]:
#parse form data from r1.content

soup1=BeautifulSoup(r1.content,'lxml')
form=soup1.find_all("form",{'id':'casLoginForm'})[0]

lt=form.find('input',{'name':'lt'})['value']
dllt=form.find('input',{'name':'dllt'})['value']
execution=form.find('input',{'name':'execution'})['value']
_eventId=form.find('input',{'name':'_eventId'})['value']
rmShown=form.find('input',{'name':'rmShown'})['value']

In [32]:
url2=r'http://cas.whu.edu.cn/authserver/login?service=http://my.whu.edu.cn/'
header2={
    'Host': 'cas.whu.edu.cn',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64; rv:54.0) Gecko/20100101 Firefox/54.0',
    'Referer':'http://cas.whu.edu.cn/authserver/login?service=http%3A%2F%2Fmy.whu.edu.cn%2F'
}

postdata={'username':username,
          'password':password,
          'lt':lt,  #these items can be get from html
          'dllt':dllt,
          'execution':execution,
          '_eventId':_eventId,
          'rmShown':rmShown
          }

There is a trick that the url 
>'/?ticket=ST-376925-aOl7OVV6iBZnl1dQXYZk1499734811083-rvbW-cas'  

is obviously a variable,we need to get it from the prior files.Usually,the path variable can be found as the value of 'Location',an item in the 'Response headers' of the prior files.It is not hard to find this variable in 
>'login?service=http%3A%2F%2Fmy.whu.edu.cn%2F'

![](./crawlerNotesFig/19.png)

***Notice**  
The [Location](https://developer.mozilla.org/en-US/docs/Web/HTTP/Headers/Location) item is a little special,to capture it,we need to set the parameter `allow_redirects` as `False` in `session.post()` method.

In [33]:
r2=session.post(url2,data=postdata,headers=header2,allow_redirects=False)
print r2.headers

{'content-length': '0', 'content-language': 'en-US', 'set-cookie': 'CASPRIVACY=""; Expires=Thu, 01-Dec-94 16:00:00 GMT; Path=/authserver/, iPlanetDirectoryPro=AQIC5wM2LY4Sfczfkf90wbg%2FQ6uu8P%2BhX%2B%2FvNiK3fHFIMEU%3D%40AAJTSQACMDI%3D%23; Path=/; Domain=.whu.edu.cn, CASTGC=TGT-179849-ROlyAbYsIcZbtIpI1p5JelQYG3IW9YOJOi2NwZb435FqMQGMWr1499745762830-vcnA-cas; Path=/authserver/; HttpOnly', 'expires': 'Thu, 01 Jan 1970 00:00:00 GMT', 'server': 'openresty', 'connection': 'keep-alive', 'location': 'http://my.whu.edu.cn/?ticket=ST-411531-dtjSVTM0qh1gmqSpSGIW1499745762873-jebj-cas', 'pragma': 'no-cache', 'cache-control': 'no-cache, no-store', 'date': 'Tue, 11 Jul 2017 04:02:42 GMT'}


Now,'Location' has been captured.  

In [34]:
r2.status_code

302

Status_code is 302,which includes a location header

In fact,there is a indirection,so,from the initial url2 we can get no content.

In [35]:
print r2.content

By the way,if we want to break the password,we can stop here and add a 'if-condition' to confirm whether or not we have found the true password.
```python
if r2.status_code==302:
    print 'the password is right'
```

Now,get back on track and we begin to crawl the target data

In [36]:
#crawl the target data
location=r2.headers['location']
url3=location
header3={
    'Host': 'my.whu.edu.cn',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64; rv:54.0) Gecko/20100101 Firefox/54.0',
    'Referer':'http://cas.whu.edu.cn/authserver/login?service=http%3A%2F%2Fmy.whu.edu.cn%2F'
}

r3=session.get(url3,headers=header3)

In [68]:
print r3.status_code

200


So great! We have login successfully.  

The remaining job is to parse the data.It is a little boring but necessary.Suppose we want to crawl the information encircled by the red line as below.
![](./crawlerNotesFig/20.png)

Again,the url
> /?.pn=p11972  

is a variable.we can find it from the html of url
> '/'  


In [39]:
#get the target html
soup3=BeautifulSoup(r3.content,'lxml')
li=soup3.find('li',text='个人信息')

urlInfo=r'http://my.whu.edu.cn/'+li.a['href']

header4={
    'Host': 'my.whu.edu.cn',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64; rv:54.0) Gecko/20100101 Firefox/54.0',
    'Referer':'http://my.whu.edu.cn/'
}


r4=session.get(urlInfo,headers=header4)

In [45]:
r4.status_code

200

Now,parse this html.

In [58]:
soup4=BeautifulSoup(r4.content)
table=soup4.find('table',{'class':'pa-main-table'})


text=table.text
text1=text.strip()
text2=text1.split('\n\n\n')

studentId=text2[0].split('\n')[1].strip()
name=text2[0].split('\n')[3].strip()
gender=text2[1].split('\n')[1].strip()
birthday=text2[1].split('\n')[3].strip()
nationality=text2[1].split('\n')[5].strip()
ploticalStatus=text2[2].split('\n')[1].strip()
idNumber=text2[2].split('\n')[3].strip()
grade=text2[3].split('\n')[1].strip()
category=text2[3].split('\n')[3].strip()
faculty=text2[3].split('\n')[5].strip()
major=text2[4].split('\n')[1].strip()
dormitory=text2[4].split('\n')[3].strip()
address=text2[4].split('\n')[5].strip()

In [62]:
print gender
print ploticalStatus
print faculty
print address

男
共青团员
经济与管理学院
湖北省武汉市武昌区珞珈山街道武汉大学


We can also get the picture

In [66]:
img=soup4.find('img',{'alt':'图片'})
urlPic=r'http://my.whu.edu.cn/'+img['src']

r5=session.get(urlPic)
with open(username+'.png','wb') as f:
    f.write(r5.content)

![](./crawlerNotesFig/21.png)